In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
#importando bs4 como classe
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
#Considera o navegador como o Chrome
driver = webdriver.Chrome()

# url do Doctoralia no Google Maps
url = 'https://www.google.com/maps/place/Doctoralia+Brasil/@-25.4346963,-49.2813171,17z/data=!4m16!1m9!3m8!1s0x94dce473651f6a7d:0x44224345aa615099!2sDoctoralia+Brasil!8m2!3d-25.4346963!4d-49.2787422!9m1!1b1!16s%2Fg%2F11d_7tzh65!3m5!1s0x94dce473651f6a7d:0x44224345aa615099!8m2!3d-25.4346963!4d-49.2787422!16s%2Fg%2F11d_7tzh65?hl=pt-BR&entry=ttu'

# url do colégio guri
#url = 'https://www.google.com/maps/place/Colegio+Guri+S%C3%AAnior/@-3.7866196,-38.5776434,17z/data=!3m1!4b1!4m6!3m5!1s0x7c74c242da07b09:0xfc31b344421f994d!8m2!3d-3.7866196!4d-38.5750685!16s%2Fg%2F11f0w_vkms?entry=ttu'

#Abre o link no Chrome e espera 20 segundos até tudo carregar
driver.get(url)
time.sleep(10)

# Clica em Avaliações
driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]').click()
time.sleep(4)


"""
# Clica em Ordenar
driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
time.sleep(2)

# Clicar em mais recentes
driver.find_element(By.XPATH, '//*[@id="action-menu"]/div[2]').click()
time.sleep(2)
"""

'\n# Clica em Ordenar\ndriver.find_element(By.XPATH, \'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button\').click()\ntime.sleep(2)\n\n# Clicar em mais recentes\ndriver.find_element(By.XPATH, \'//*[@id="action-menu"]/div[2]\').click()\ntime.sleep(2)\n'

In [3]:
# Calcula o número total de avaliações feitqs
qtd_reviews = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0]

# Converte o texto cobtendo o nýumero em formato de número
if ',' in qtd_reviews:
    qtd_reviews = int(qtd_reviews.replace(',',''))
elif '.' in qtd_reviews:
    qtd_reviews = int(qtd_reviews.replace('.',''))
else: 
    qtd_reviews = int(qtd_reviews)

In [4]:
#Encontra onde fazer a rolagem
barra_rolagem = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

#Faz a rolagem com base na quantidade de avaliações feitas
for i in range(0,round(qtd_reviews/5)+1):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', barra_rolagem)
    time.sleep(2)

In [5]:
"""
# Encontra os botões de "Mais" para expandir os comentários
botoes_mais = driver.find_elements(By.CLASS_NAME,"w8nwRe kyuRq")

# Clica no botão de "Mais" de cada comentário para expandir cada comentário
for botao_mais in botoes_mais:
    botao_mais.click()
    time.sleep(2)
"""

'\n# Encontra os botões de "Mais" para expandir os comentários\nbotoes_mais = driver.find_elements(By.CLASS_NAME,"w8nwRe kyuRq")\n\n# Clica no botão de "Mais" de cada comentário para expandir cada comentário\nfor botao_mais in botoes_mais:\n    botao_mais.click()\n    time.sleep(2)\n'

In [6]:
# Coleta o html da pagina 
resposta = BeautifulSoup(driver.page_source, 'html.parser')

# Fecha o navegador controlado pela automação
driver.close()

# Cria uma lista com o código html de cada comentário (Nome, nota, período, review, links, fotos etc)
reviews = resposta.find_all('div', class_='jftiEf fontBodyMedium')

In [7]:
# Função que cria um dicionário com o conjunto de cada nome, nota, período e review
def coleta_reviews(result_set):
    rev_dict = {
        'Nome': [],
        'Nota': [],
        'Periodo': [],
        'Review' : []
    }
    for result in result_set:
        nome = result.find('div', class_='d4r55').text # encontra o código referente ao nome do usuário e salva em uma variável
        nota = result.find('span', class_='kvMYJc')["aria-label"].split(" ")[0] # encontra o código referente à nota do usuário e salva em uma variável
        periodo = result.find('span',class_='rsqaWe').text # encontra o código referente ao período do usuário e salva em uma variável
        #Tenta (try) encontrar o comentário e salvar em uma váriável, caso (except) não encontre, salva nulo em uma váriável
        try:
            review = result.find('span',class_='wiI7pd').text
        except:
            review = np.nan
        
        # salva cada conjunto de nome, nota, período e review no dicionário
        rev_dict['Nome'].append(nome)
        rev_dict['Nota'].append(nota)
        rev_dict['Periodo'].append(periodo)
        rev_dict['Review'].append(review)
    
    return(pd.DataFrame(rev_dict)) # retorna um dataframe do pandas criado a partir do dicionário

df_reviews = coleta_reviews(reviews)

In [8]:
df_reviews

,Nome,Nota,Periodo,Review
0,Mayara Thays Beckhauser,1,3 meses atrás,"Faço das palavras das muitas reclamações, as m..."
1,Wendy Evangelista,1,2 semanas atrás,Péssimo para os profissionais que contratam!!!...
2,Lidia Izeppe,1,5 meses atrás,Só pense muuuuito bem antes de assinar o contr...
3,Ana Vitoria,1,5 meses atrás,diz que o medico atende para meu plano de saúd...
4,Gertrudes Boska,5,7 meses atrás,Como não encontrei um local especifico para a ...
...,...,...,...,...
1135,Evandro Santos,5,3 anos atrás,NaN
1136,Nadine Linard,4,4 anos atrás,NaN
1137,Patricia Cardoso,5,3 anos atrás,NaN
1138,José Mário Espindola,4,3 anos atrás,NaN


In [9]:
# Coleta a lista de nomes do censo de 2010
#dados disponíveis em https://brasil.io/dataset/genero-nomes/nomes/
nomes_censo = pd.read_csv('grupos.csv', sep = ',')

In [10]:
#Cria um dicionário de cada possibilidade de nome disponível no censo e a classificação de sexo
classification_list = {}
for i in nomes_censo['names']:
    names_list = i.split("|")
    for j in names_list:
        if j != '':
            classification_list[j] = nomes_censo.loc[nomes_censo['names'] == i,'classification'].values[0]

In [11]:
# Cria uma coluna no dataframe do pandas com o primeiro nome coletado dos reviews do google
# e coloca no formato do censo de 2010
from unicodedata import normalize

df_reviews['Prim_nome_normalizado'] = df_reviews['Nome'].str.normalize('NFKD').str.encode("ascii", errors="ignore").str.decode("ascii").str.upper().str.split().str[0]

In [12]:
# Cria uma coluna com a classificação do sexo a partir do primeiro nome de cada usuário 
# e compara com o dicionário disponível no censo de 2010. Caso não encontre o nome, retorna S.I
df_reviews['Sexo'] = df_reviews['Prim_nome_normalizado'].map(classification_list).fillna('S.I.')

In [13]:
# Cria uma coluna padronizando os períodos de avaliação para dias
periodo_padronizado_dias = []

for i in df_reviews["Periodo"]:
    if i == 'um dia atrás':
        periodo_padronizado_dias.append(1)
    elif i == 'uma semana atrás':
        periodo_padronizado_dias.append(7)
    elif i == 'um mês atrás':
        periodo_padronizado_dias.append(30)
    elif i == 'um ano atrás':
        periodo_padronizado_dias.append(365)
    elif i.split()[1] == 'dias':
        periodo_padronizado_dias.append(int(i.split()[0]))    
    elif i.split()[1] == 'semanas':
        periodo_padronizado_dias.append(int(i.split()[0])*7)
    elif i.split()[1] == 'meses':
        periodo_padronizado_dias.append(int(i.split()[0])*30)
    elif i.split()[1] == 'anos':
        periodo_padronizado_dias.append(int(i.split()[0])*365)
    else:
        periodo_padronizado_dias.append(0)

df_reviews['Periodo_padronizado_dias'] = periodo_padronizado_dias

In [14]:
df_reviews

,Nome,Nota,Periodo,Review,Prim_nome_normalizado,Sexo,Periodo_padronizado_dias
0,Mayara Thays Beckhauser,1,3 meses atrás,"Faço das palavras das muitas reclamações, as m...",MAYARA,F,90
1,Wendy Evangelista,1,2 semanas atrás,Péssimo para os profissionais que contratam!!!...,WENDY,F,14
2,Lidia Izeppe,1,5 meses atrás,Só pense muuuuito bem antes de assinar o contr...,LIDIA,F,150
3,Ana Vitoria,1,5 meses atrás,diz que o medico atende para meu plano de saúd...,ANA,F,150
4,Gertrudes Boska,5,7 meses atrás,Como não encontrei um local especifico para a ...,GERTRUDES,F,210
...,...,...,...,...,...,...,...
1135,Evandro Santos,5,3 anos atrás,NaN,EVANDRO,M,1095
1136,Nadine Linard,4,4 anos atrás,NaN,NADINE,F,1460
1137,Patricia Cardoso,5,3 anos atrás,NaN,PATRICIA,F,1095
1138,José Mário Espindola,4,3 anos atrás,NaN,JOSE,M,1095


In [15]:
# Salva o arquivo 
df_reviews[['Nome','Sexo','Nota','Periodo','Periodo_padronizado_dias','Review']].to_excel("df_reviews.xlsx",sheet_name='reviews', index=False)